In [ ]:
import random

import numpy as np
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

In [ ]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat


def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat


def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat


def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat


def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks)

    return feat


def feature_extraction(X):
    #Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    for sample in X:
        features = A(sample)
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        #features = np.hstack((features, TBP(sample)))
        X_tmp.append(features)

    X = np.array(X_tmp)
    return X

In [ ]:
np.random.seed(42)
data_input_file = "/home/jsenadesouza/DA-healthy2patient/results/pain/dataset/pain_patientid_acc_f100_t5_kfold.npz"
feature_file = data_input_file.split('.npz')[0] + '_features.npz'
#     np.savez_compressed(data_input_file.split('.npz')[0] + '_features.npz', X=X, y=y, folds=folds)

print("Extracting features")
tmp = np.load(data_input_file, allow_pickle=True)
X = tmp['X']
y = tmp['y']
folds = tmp['folds']
X = np.transpose(np.squeeze(X), (0, 2, 1))
y_label, y_id = [], []
for yy in y:
    y_label.append(yy.split("_")[0])
    y_id.append(yy.split("_")[1])

y = np.array(y_label)

# Undersampling the data to balance the classes
rus = RandomUnderSampler(random_state=42, sampling_strategy={'none': 1000, 'mild': 1000, 'moderate': 1000, 'severe': 1000})
rus.fit_resample(X[:, :, 0], y)
X, y = X[rus.sample_indices_], y[rus.sample_indices_]
print("Undersampling done")

X_feat = feature_extraction(X)

In [ ]:
df = pd.DataFrame(np.hstack((np.squeeze(X_feat), np.expand_dims(y,axis=1))), columns=['Ax', 'Ay', 'Az', 'SDx','SDy','SDz', 'AADx', 'AADy', 'AADz', 'ARA', 'label'])
df = df.astype(dtype= {'Ax':"float64", 'Ay':"float64", 'Az':"float64", 'SDx':"float64",'SDy':"float64",'SDz':"float64", 'AADx':"float64", 'AADy':"float64", 'AADz':"float64", 'ARA':"float64", 'label':'object'})

In [ ]:
df_non = df[df['label'] == 'none']
df_mild = df[df['label'] == 'mild']
df_moderate = df[df['label'] == 'moderate']
df_severe = df[df['label'] == 'severe']


In [ ]:
pd.DataFrame([df_non.mean(axis=0), df_mild.mean(axis=0), df_moderate.mean(axis=0), df_severe.mean(axis=0)],
             index=['none', 'mild', 'moderate', 'severe']).T

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

def interpolate_sensors(samples, type_interp, n_samples, plot=False):
    samples = np.array(samples)
    shape = samples.shape
    if len(shape) > 3:
        samples = np.squeeze(samples)
        shape = samples.shape
    new_samples = []
    for smp in samples:
        s = []
        for sig in range(shape[-1]):
            size = int(smp.shape[0])
            x = np.linspace(0, size-1, num=size, endpoint=True)
            y = smp[:,sig]
            f = interp1d(x, y, kind= type_interp)
            x_new = np.linspace(0, size-1, num=n_samples, endpoint=True)
            y_new = f(x_new)
            s.append(y_new)
            if plot:
                plt.plot(x, y, '--', x_new, y_new, 'o')
                plt.xticks(x_new)
                plt.show()
        s = np.array(s)
        new_samples.append(s)
    #new_samples = np.array(new_samples)
    return new_samples

def plot_acceleration(sample, timestamp):

    fig, ax = plt.subplots(figsize=(10,6))
    time = pd.to_datetime(timestamp, format='%Y-%m-%d %H:%M:%S.%f')
  # add every single subplot to the figure with a for loop
    sns.lineplot(time, sample[:, 0], label='x', ax=ax)
    sns.lineplot(time, sample[:, 1], label='y', ax=ax)
    sns.lineplot(time, sample[:, 2], label='z', ax=ax)
    #ax.title.set_text(label + '\n' + str(v))
    #ax.set_ylim(-0.5,0.5)
    #ax.legend()
    #plt.axis('equal')
    plt.show()

In [ ]:
df = pd.read_csv('/home/jsenadesouza/DA-healthy2patient/data/clinical_data/pain_acc/pain_30minmargin_measurednowcol_p_44_t_2018-03-27_bp_WRIST.csv')
df.head()

In [ ]:
df_day = df[df['timestamp'].str.contains('2018-03-28')]
df_day.tail()

In [ ]:
df_day = df[df['timestamp'].str.contains('2018-03-29')]
df_day.tail()

In [ ]:
np.unique(df_day['pain_score'])

In [ ]:
plot_acceleration(df_day[['x','y','z']].values)

In [ ]:
df_day[['x','y','z']].values.shape

In [ ]:
df_frame = df[:3527999]

In [ ]:
plot_acceleration(df_frame[['x','y','z']].values, df_frame['timestamp'])


In [ ]:
intervals = []
last = df_frame['pain_score'].iloc[0]
intervals.append({'pain_score': last, 'start': 0, 'end': 0})
for i in tqdm(range(1, len(df_frame))):
    if df_frame.iloc[i]['pain_score'] != last:
        intervals[-1]["end"] = i-1
        intervals.append({'pain_score': df_frame.iloc[i]['pain_score'], 'start': i, 'end': i})
    last = df_frame.iloc[i]['pain_score']